In [ ]:
# =====================================
# 🛠 2. Import Required Modules
# =====================================
import plotly.graph_objects as go
import ta
from breeze_connect import BreezeConnect
import pandas as pd
import numpy as np
import json
from datetime import datetime, timedelta
import os
from app_config import Config
from breeze_utils import BreezeDataManager
from model_utils import create_time_features, calculate_technical_indicators

# =====================================
# 🛠 3. Initialize Secure Data Manager
# =====================================
# Initialize secure configuration and data manager
config = Config()
breeze_manager = BreezeDataManager()

# Set up Google Drive if in Colab environment
try:
    from google.colab import drive
    drive.mount('/content/drive')
    print("✅ Google Drive mounted")
except ImportError:
    print("ℹ️ Not in Colab environment, skipping Google Drive mount")

# Authenticate with Breeze API
if breeze_manager.authenticate():
    print("✅ Breeze API authenticated successfully")
    breeze = breeze_manager.breeze
else:
    print("❌ Failed to authenticate with Breeze API")
    raise Exception("Authentication failed")

# =====================================
# 🛠 4. Helper Functions (Now Using Utilities)
# =====================================

# 📅 Date formatters
def get_date_iso(days_ago=0):
    date = datetime.now() - timedelta(days=days_ago)
    date = date.replace(hour=9, minute=0, second=0, microsecond=0)
    return date.isoformat() + ".000Z"

def get_end_date_iso():
    date = datetime.now().replace(hour=15, minute=30, second=0, microsecond=0)
    return date.isoformat() + ".000Z"

# def get_last_trading_day(days_back=0):
#     """Returns the last trading day (skips weekends and holidays)"""
#     date = datetime.now() - timedelta(days=days_back)
#     attempts = 0
#     max_attempts = 10  # To prevent infinite loops

#     while attempts < max_attempts:
#         # Skip weekends
#         if date.weekday() >= 5:  # 5=Saturday, 6=Sunday
#             date -= timedelta(days=1)
#             attempts += 1
#             continue

#         # Verify trading day by checking if we can get index data
#         try:
#             response = breeze.get_historical_data_v2(
#                 interval="1day",
#                 from_date=date.replace(hour=9, minute=0, second=0, microsecond=0).isoformat() + ".000Z",
#                 to_date=date.replace(hour=15, minute=30, second=0, microsecond=0).isoformat() + ".000Z",
#                 stock_code="NIFTY",
#                 exchange_code="NSE",
#                 product_type="cash"
#             )
#             if response['Status'] == 200:
#                 return date
#         except:
#             pass

#         date -= timedelta(days=1)
#         attempts += 1

#     return datetime.now()  # Fallback to today if can't determine

def detect_strike_step(strikes):
    if len(strikes) < 2:
        return None
    return int(min(np.diff(sorted(strikes))))

def get_nearest_strike_price(ltp, valid_step):
    if ltp:
      return int(round(ltp / valid_step) * valid_step)
    else:
      return None

# 📈 Fetch LTP
def get_live_ltp(stock_code, exchange_code):
    response = breeze.get_quotes(stock_code=stock_code, exchange_code=exchange_code)
    if response['Status'] == 200:
        ltp = float(response['Success'][0]['ltp'])
        print(f"📦 Last Traded Price for {stock_code} is {ltp}")
        return ltp
    else:
        print(f"❌ Error fetching LTP: {response['Error']}")
        return None

# 📈 Fetch Historical Data
def fetch_historical_data(stock_code, exchange_code, product_type, interval, from_date, to_date, file_name, expiry_date=None, strike_price=None, right=None):
    print(f"Fetching {product_type.upper()} data for {stock_code} with Strike Price = {strike_price}")

    params = {
        "interval": interval,
        "from_date": from_date,
        "to_date": to_date,
        "stock_code": stock_code,
        "exchange_code": exchange_code,
        "product_type": product_type
    }

    if product_type == "futures" and expiry_date:
        params["expiry_date"] = expiry_date
        print(f"Fecting data for FUTURE with Expiry: {expiry_date}")

    if product_type == "options":
        if not all([expiry_date, strike_price, right]):
            print("❌ Missing expiry/strike/right for options.")
            return
        params["expiry_date"] = expiry_date
        params["strike_price"] = strike_price
        params["right"] = right.lower()
        print(f"Fecting data for OPTIONS for Strike Price:{strike_price} with Expiry: {expiry_date}")

    response = breeze.get_historical_data_v2(**params)

    if response['Status'] == 200:
        data = pd.DataFrame(response['Success'])
        data.to_csv(os.path.join(save_path, file_name), index=False)
        print(f"✅ Data saved to {file_name}")
    else:
        print(f"❌ Error fetching data: {response['Error']}")

# 📈 Simple Trend Detection (Bullish/Bearish)
def detect_trend(stock_code, exchange_code, lookback_days=7):
    from_date = get_last_trading_day(days_back=lookback_days)
    to_date = get_last_trading_day(days_back=0)

    from_date_iso = from_date.replace(hour=9, minute=0, second=0, microsecond=0).isoformat() + ".000Z"
    to_date_iso = to_date.replace(hour=15, minute=30, second=0, microsecond=0).isoformat() + ".000Z"

    candles = breeze.get_historical_data_v2(
        interval="1day",
        from_date=from_date_iso,
        to_date=to_date_iso,
        stock_code=stock_code,
        exchange_code=exchange_code,
        product_type="cash"
    )

    if candles['Status'] != 200:
        print(f"❌ Error fetching trend data: {candles['Error']}")
        return None

    df = pd.DataFrame(candles['Success'])
    df['open'] = df['open'].astype(float)
    df['close'] = df['close'].astype(float)

    trend_open = df.iloc[0]['open']
    trend_close = df.iloc[-1]['close']

    if trend_close > trend_open:
        print(f"📈 Bullish Trend: Open={trend_open} ➡️ Close={trend_close}")
        return "call"
    else:
        print(f"📉 Bearish Trend: Open={trend_open} ➡️ Close={trend_close}")
        return "put"

def get_actual_expiry_dates(stock_code, max_weeks=4):
    """Get valid expiry dates by checking historical data availability"""
    today = datetime.today()
    valid_expiries = []

    for i in range(max_weeks):
        days_ahead = ((3 - today.weekday()) % 7) + (i * 7)  # Next Thursdays
        potential_expiry = today + timedelta(days=days_ahead)
        expiry_str = potential_expiry.strftime('%Y-%m-%d')

        print(f"🔍 Checking expiry: {expiry_str}")

        try:
            # Check using historical data API which is available in 1.0.62
            response = breeze.get_historical_data_v2(
                interval="30minute",
                from_date=get_last_trading_day(5).strftime('%Y-%m-%d') + "T09:00:00.000Z",
                to_date=get_last_trading_day(0).strftime('%Y-%m-%d') + "T15:45:00.000Z",
                stock_code=stock_code,
                exchange_code="NFO",
                product_type="futures",
                expiry_date=expiry_str
            )

            if response['Status'] == 200 and response['Success']:
                valid_expiries.append(expiry_str)
                print(f"✅ Valid expiry found: {expiry_str}")
        except Exception as e:
            print(f"⚠️ Error checking expiry {expiry_str}: {str(e)}")

    return valid_expiries

def get_next_valid_expiry(stock_code, product_type="futures", max_weeks=4):
    """Find next valid expiry using available methods"""
    valid_expiries = get_actual_expiry_dates(stock_code, max_weeks)

    if not valid_expiries:
        print(f"❌ No valid expiry found in next {max_weeks} weeks")
        return None

    # Return nearest expiry
    nearest_expiry = valid_expiries[0]
    print(f"📌 Using expiry: {nearest_expiry}")
    return nearest_expiry

def get_valid_option_strikes(stock_code, expiry_date):
    """Get all valid strike prices for a given expiry"""
    if not expiry_date:
        print("❌ No expiry date provided")
        return []

    try:
        # First get LTP to estimate reasonable strikes
        ltp = get_live_ltp(stock_code, "NSE")
        if not ltp:
            print("❌ Couldn't get LTP for strike estimation")
            return []

        # Get strikes for both calls and puts near the money
        all_strikes = set()

        for right in ['call', 'put']:
            # Get chain for this right with estimated ATM strike
            atm_strike = get_nearest_strike_price(ltp, 20)  # Assuming 50 strike interval

            chain_response = breeze.get_option_chain_quotes(
                stock_code=stock_code,
                exchange_code="NFO",
                product_type="options",
                expiry_date=expiry_date + "T06:00:00.000Z",
                right=right
            )

            if chain_response['Status'] == 200:
                for item in chain_response.get('Success', []):
                    try:
                        all_strikes.add(float(item['strike_price']))
                    except (KeyError, ValueError):
                        continue

        if not all_strikes:
            print("⚠️ No strikes found in option chain")
            return []

        print(f"✅ Found {len(all_strikes)} strikes for {expiry_date}")
        return sorted(all_strikes)

    except Exception as e:
        print(f"❌ Exception in get_valid_option_strikes: {str(e)}")
        return []

def fetch_all_options_data(stock_code, exchange_code, expiry_date, ltp, interval, from_date, to_date, strike_range=800):
    """Fetch all options within range using SDK 1.0.62 compatible methods"""
    if not expiry_date:
        print("❌ No expiry date provided")
        return None

    print(f"\n📦 Fetching ALL options data for {stock_code} expiry {expiry_date}")
    print(f"   Strike Range: {ltp-strike_range} to {ltp+strike_range}")

    all_options_data = []

    # Determine strike step (TCS usually 20, NIFTY 50)
    strike_step = 20 if stock_code == "TCS" else 50

    # Calculate nearest strike
    atm_strike = round(ltp / strike_step) * strike_step

    # Generate strikes in range
    min_strike = atm_strike - strike_range
    max_strike = atm_strike + strike_range
    strikes = range(
        int(min_strike // strike_step * strike_step),
        int(max_strike // strike_step * strike_step) + strike_step,
        strike_step
    )

    print(f"🔢 Fetching {len(strikes)} strikes from {min(strikes)} to {max(strikes)}")

    for strike in strikes:
        for right in ['call', 'put']:
            print(f"🔄 Fetching {right.upper()} {strike}...", end=' ')
            try:
                response = breeze.get_historical_data_v2(
                    interval=interval,
                    from_date=from_date,
                    to_date=to_date,
                    stock_code=stock_code,
                    exchange_code=exchange_code,
                    product_type="options",
                    expiry_date=expiry_date,
                    strike_price=strike,
                    right=right
                )

                if response['Status'] == 200:
                    df = pd.DataFrame(response['Success'])
                    if not df.empty:
                        df['strike'] = strike
                        df['right'] = right
                        df['expiry_date'] = expiry_date
                        all_options_data.append(df)
                        print(f"✅ {right.upper()} {strike} fetched ({len(df)} records)")
                    else:
                        print("⚠️ No data")
                else:
                    print(f"❌ Failed to fetch {right.upper()} {strike}: {response['Error']}")
            except Exception as e:
                print(f"❌ Error fetching {right.upper()} {strike}: {str(e)}")

    if all_options_data:
        combined_df = pd.concat(all_options_data, ignore_index=True)

        # Add derived columns
        combined_df['datetime'] = pd.to_datetime(combined_df['datetime'])
        combined_df['date'] = combined_df['datetime'].dt.date
        combined_df['time'] = combined_df['datetime'].dt.time

        expiry_date_str = pd.to_datetime(expiry_date).strftime('%d%b%y').upper()
        combined_df['symbol'] = (
            stock_code +
            expiry_date_str +
            combined_df['strike'].astype(int).astype(str) +
            combined_df['right'].str[0].str.upper()
        )

        print("\n🔢 Normalizing records across all options...")

        # 1. Find the most complete option (with most timestamps)
        option_counts = combined_df.groupby(['strike', 'right']).size()
        max_records = option_counts.max()
        print(f"Maximum records for any option: {max_records}")

        # 2. Get the complete set of timestamps from the most complete option
        complete_option = option_counts.idxmax()
        complete_timestamps = combined_df[
            (combined_df['strike'] == complete_option[0]) &
            (combined_df['right'] == complete_option[1])
        ]['datetime'].sort_values().unique()

        # 3. Normalize each option to these timestamps
        normalized_dfs = []

        for (strike, right), group in combined_df.groupby(['strike', 'right']):
            # Create complete index for this option
            option_df = pd.DataFrame({'datetime': complete_timestamps})

            # Merge with actual data
            option_df = option_df.merge(
                group,
                on='datetime',
                how='left'
            )

            # Fill in static columns
            option_df['strike'] = strike
            option_df['right'] = right
            option_df['expiry_date'] = expiry_date
            option_df['symbol'] = f"{stock_code}{expiry_date_str}{strike}{right[0].upper()}"

            # Re-add date/time columns
            option_df['date'] = option_df['datetime'].dt.date
            option_df['time'] = option_df['datetime'].dt.time

            normalized_dfs.append(option_df)

        # Combine all normalized data
        normalized_df = pd.concat(normalized_dfs, ignore_index=True)

        # Calculate statistics
        num_options = len(normalized_df['symbol'].unique())
        records_per_option = len(normalized_df) / num_options

        print(f"✅ Normalized to {len(normalized_df)} total records")
        print(f"   {records_per_option:.0f} records per option ({num_options} options)")

        # Save to file
        file_path = os.path.join(save_path, "tcs_options_data.csv")
        normalized_df.to_csv(file_path, index=False)
        print(f"\n✅✅ ALL options data saved to {file_path}")
        print(f"   Total Records: {len(normalized_df)}")
        print(f"   From: {normalized_df['datetime'].min()} to {normalized_df['datetime'].max()}")
        return normalized_df
    else:
        print("⚠️ No options data was fetched")
        return None

# =====================================
# 🛠 5. Parameter Setup
# =====================================

# Use methods from BreezeDataManager for date operations
get_date_iso = breeze_manager.get_date_iso
get_end_date_iso = breeze_manager.get_end_date_iso
get_last_trading_day = lambda days_back=0: breeze_manager.get_last_trading_day(days_back)

print("✅ All modules and utilities loaded successfully!")

stock_name = "TCS"
interval = "5minute"

# Use trading-day aware dates
last_trading_day = get_last_trading_day(0)

from_date = get_last_trading_day(30).strftime('%Y-%m-%d') + "T09:00:00.000Z"
to_date = last_trading_day.strftime('%Y-%m-%d') + "T15:30:00.000Z"

# Get LTP
# ltp = get_live_ltp(stock_code=stock_name, exchange_code="NSE")

# ✅ 2. Assume default strike step (TCS usually has 20)
default_strike_step = 20 if stock_name.upper() == "TCS" else 100 # Or make a mapping

# Detect based on past week
option_type = detect_trend(stock_code=stock_name, exchange_code="NSE", lookback_days=7) # "call" or "put"
# Or for past month
# option_type = detect_trend(stock_code=stock_name, exchange_code="NSE", lookback_days=30)

expiry_date_str = get_next_valid_expiry(stock_name)
if not expiry_date_str:
    print("❌ Could not determine valid expiry date")
    exit()

ltp = get_live_ltp(stock_code=stock_name, exchange_code="NSE")
if not ltp:
    print("❌ Could not get LTP")
    exit()

# Now get valid strikes for options
valid_strikes = get_valid_option_strikes(stock_name, expiry_date_str)

if not valid_strikes:
    print("⚠️ Could not get option strikes. Using default strike step.")
    strike_step = default_strike_step
    strike_price = get_nearest_strike_price(ltp, strike_step)
    valid_strikes = [strike_price]  # Fallback to just this strike
else:
    strike_step = detect_strike_step(valid_strikes) or default_strike_step
    strike_price = get_nearest_strike_price(ltp, strike_step)

print(f"📌 Final parameters: {expiry_date_str=}, {strike_price=}, {valid_strikes=}")

# =====================================
# 🛠 6. Fetch and Save Data
# =====================================

# 1️⃣ Fetch Equity
fetch_historical_data(
    stock_code=stock_name,
    exchange_code="NSE",
    product_type="cash",
    interval=interval,
    from_date=from_date,
    to_date=to_date,
    file_name="tcs_equity_data.csv"
)

# 2️⃣ Fetch Futures
futures_expiry_str = get_next_valid_expiry(
    stock_code=stock_name,
    product_type="futures"
)

if futures_expiry_str:
    fetch_historical_data(
        stock_code=stock_name,
        exchange_code="NFO",
        product_type="futures",
        interval=interval,
        from_date=from_date,
        to_date=to_date,
        file_name="tcs_futures_data.csv",
        expiry_date=futures_expiry_str
    )

# 3️⃣ Fetch Options (Auto Strike)
# Fetch full option chain instead of one strike
options_df = fetch_all_options_data(
    stock_code=stock_name,
    exchange_code="NFO",
    expiry_date=expiry_date_str,
    ltp=ltp,
    interval=interval,
    from_date=from_date,
    to_date=to_date,
    strike_range=800
)

if options_df is None:
    print("⚠️ Warning: No options data was fetched")

# 🎯 Done
print("✅✅✅ All data fetched and saved to Google Drive successfully!")

# =====================================
# 🛠 7. Technical Indicator Generator
# =====================================
def add_all_technical_indicators(df):
    df = df.copy()
    
    # Ensure numeric columns
    numeric_cols = ['open', 'high', 'low', 'close', 'volume']
    for col in numeric_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce').ffill()
    
    # ===== TREND INDICATORS =====
    # ADX Family
    try:
        df['ADX_14'] = ta.trend.adx(df['high'], df['low'], df['close'])
        df['ADX_pos'] = ta.trend.adx_pos(df['high'], df['low'], df['close'])
        df['ADX_neg'] = ta.trend.adx_neg(df['high'], df['low'], df['close'])
    except Exception as e:
        print(f"ADX Error: {str(e)}")

    # Aroon
    try:
        df['Aroon_Up'] = ta.trend.aroon_up(df['high'], df['low'])
        df['Aroon_Down'] = ta.trend.aroon_down(df['high'], df['low'])
        df['Aroon_Osc'] = df['Aroon_Up'] - df['Aroon_Down']
    except Exception as e:
        print(f"Aroon Error: {str(e)}")

    # MACD
    try:
        macd = ta.trend.MACD(df['close'])
        df['MACD'] = macd.macd()
        df['MACD_signal'] = macd.macd_signal()
        df['MACD_diff'] = macd.macd_diff()
        df['MACD_hist'] = df['MACD'] - df['MACD_signal']
    except Exception as e:
        print(f"MACD Error: {str(e)}")

    # Ichimoku Cloud
    try:
        ichimoku = ta.trend.IchimokuIndicator(df['high'], df['low'])
        df['Ichimoku_conv'] = ichimoku.ichimoku_conversion_line()
        df['Ichimoku_base'] = ichimoku.ichimoku_base_line()
        df['Ichimoku_a'] = ichimoku.ichimoku_a()
        df['Ichimoku_b'] = ichimoku.ichimoku_b()
    except Exception as e:
        print(f"Ichimoku Error: {str(e)}")

    # Moving Averages
    ma_windows = [5, 10, 20, 50, 100, 200]
    for window in ma_windows:
        try:
            df[f'SMA_{window}'] = ta.trend.sma_indicator(df['close'], window=window)
        except Exception as e:
            print(f"SMA_{window} Error: {str(e)}")
        
        try:
            df[f'EMA_{window}'] = ta.trend.ema_indicator(df['close'], window=window)
        except Exception as e:
            print(f"EMA_{window} Error: {str(e)}")
        
        try:
            df[f'WMA_{window}'] = ta.trend.wma_indicator(df['close'], window=window)
        except Exception as e:
            print(f"WMA_{window} Error: {str(e)}")

    # ===== MOMENTUM INDICATORS =====
    # RSI Family
    rsi_windows = [7, 14, 21, 28]
    for window in rsi_windows:
        try:
            df[f'RSI_{window}'] = ta.momentum.rsi(df['close'], window=window)
        except Exception as e:
            print(f"RSI_{window} Error: {str(e)}")

    # Stochastic
    try:
        stoch = ta.momentum.StochasticOscillator(df['high'], df['low'], df['close'])
        df['Stoch_%K'] = stoch.stoch()
        df['Stoch_%D'] = stoch.stoch_signal()
        df['Stoch_RSI'] = ta.momentum.stochrsi(df['close'])
    except Exception as e:
        print(f"Stochastic Error: {str(e)}")

    # Other Momentum
    momentum_indicators = [
        ('CCI', lambda: ta.trend.cci(df['high'], df['low'], df['close'])),
        ('DPO', lambda: ta.trend.dpo(df['close'])),
        ('KST', lambda: ta.trend.kst(df['close'])),
        ('KST_sig', lambda: ta.trend.kst_sig(df['close'])),

        ('PSAR_up', lambda: ta.trend.psar_up(df['high'], df['low'], df['close'])),
        ('PSAR_down', lambda: ta.trend.psar_down(df['high'], df['low'], df['close'])),
        ('PSAR_up_indicator', lambda: ta.trend.psar_up_indicator(df['high'], df['low'], df['close'])),
        ('PSAR_down_indicator', lambda: ta.trend.psar_down_indicator(df['high'], df['low'], df['close'])),
        ('TRIX', lambda: ta.trend.trix(df['close'])),

        ('TSI', lambda: ta.momentum.tsi(df['close'])),
        ('ROC', lambda: ta.momentum.roc(df['close'])),
        ('PPO', lambda: ta.momentum.ppo(df['close'])),
        ('PVO', lambda: ta.momentum.pvo(df['volume'])),
        ('KAMA', lambda: ta.momentum.kama(df['close'])),
        ('WILLR', lambda: ta.momentum.williams_r(df['high'], df['low'], df['close']))
    ]
    
    for name, func in momentum_indicators:
        try:
            df[name] = func()
        except Exception as e:
            print(f"{name} Error: {str(e)}")

    # ===== VOLATILITY INDICATORS =====
    try:
        df['ATR_14'] = ta.volatility.average_true_range(df['high'], df['low'], df['close'])
    except Exception as e:
        print(f"ATR/NATR Error: {str(e)}")

    # Bollinger Bands
    try:
        bb = ta.volatility.BollingerBands(df['close'])
        df['BB_MAVG'] = bb.bollinger_mavg()
        df['BB_HIGH'] = bb.bollinger_hband()
        df['BB_LOW'] = bb.bollinger_lband()
        df['BB_WIDTH'] = bb.bollinger_wband()
        df['BB_PERCENT'] = bb.bollinger_pband()
    except Exception as e:
        print(f"Bollinger Bands Error: {str(e)}")

    # Donchian Channel
    try:
        df['DC_HIGH'] = ta.volatility.donchian_channel_hband(df['high'], df['low'], df['close'])
        df['DC_LOW'] = ta.volatility.donchian_channel_lband(df['high'], df['low'], df['close'])
        df['DC_MID'] = (df['DC_HIGH'] + df['DC_LOW']) / 2
    except Exception as e:
        print(f"Donchian Channel Error: {str(e)}")

    # ===== VOLUME INDICATORS =====
    volume_indicators = [
        ('OBV', lambda: ta.volume.on_balance_volume(df['close'], df['volume'])),
        ('CMF', lambda: ta.volume.chaikin_money_flow(df['high'], df['low'], df['close'], df['volume'])),
        ('MFI', lambda: ta.volume.money_flow_index(df['high'], df['low'], df['close'], df['volume'])),
        ('ADI', lambda: ta.volume.acc_dist_index(df['high'], df['low'], df['close'], df['volume'])),
        ('EOM', lambda: ta.volume.ease_of_movement(df['high'], df['low'], df['volume'])),
        ('VWAP', lambda: ta.volume.volume_weighted_average_price(df['high'], df['low'], df['close'], df['volume'])),
        ('FI', lambda: ta.volume.force_index(df['close'], df['volume']))
    ]
    
    for name, func in volume_indicators:
        try:
            df[name] = func()
        except Exception as e:
            print(f"{name} Error: {str(e)}")

    # Volume Moving Averages
    for window in [5, 10, 20, 50]:
        try:
            df[f'Volume_MA_{window}'] = df['volume'].rolling(window).mean()
        except Exception as e:
            print(f"Volume_MA_{window} Error: {str(e)}")

    # ===== CUSTOM INDICATORS =====
    try:
        # Price * Volume
        df['PV'] = df['close'] * df['volume']
        
        # Returns
        df['Daily_Return'] = ta.others.daily_return(df['close'])
        df['Cum_Return'] = ta.others.cumulative_return(df['close'])
        df['Log_Return'] = np.log(df['close'] / df['close'].shift(1))
        
        # Price Change
        df['Price_Change'] = df['close'].diff()
        df['Pct_Change'] = df['close'].pct_change(fill_method=None)
        
        # Volatility
        df['HL_Pct'] = (df['high'] - df['low']) / df['low'] * 100
        df['OC_Pct'] = (df['close'] - df['open']) / df['open'] * 100
    except Exception as e:
        print(f"Custom Indicators Error: {str(e)}")

    return df

# =====================================
# 🛠 8. After Download — Read CSVs & Process
# =====================================

def filter_trading_hours(df):
    df['datetime'] = pd.to_datetime(df['datetime'])
    df = df.set_index('datetime')
    df = df.between_time("09:15", "15:30").reset_index()
    return df

def process_options_data(df):
    # Validate required columns
    if not all(col in df.columns for col in ['strike', 'right']):
        raise ValueError("DataFrame must contain 'strike' and 'right' columns")
    
    # Group and apply indicators without operating on grouping columns
    options_df = df.copy()
    options_df = options_df.groupby(['strike', 'right'], group_keys=False)\
                           .apply(lambda x: add_all_technical_indicators(x), include_groups=False).reset_index(drop=True)

    print(f"Processed options data with {len(options_df.columns)} indicators")

    return options_df

# 🧹 Helper to load
def load_and_process(file_path):
    if not os.path.exists(file_path) or os.stat(file_path).st_size == 0:
        print(f"⚠️ Skipping {file_path} — File is missing or empty.")
        return None
    df = pd.read_csv(file_path)
    df = filter_trading_hours(df)

    if 'strike' in df.columns and 'right' in df.columns:
        return process_options_data(df)
    else:
        df = add_all_technical_indicators(df)
        print(f"{file_path} finally has columns: {df.columns}")
    return df

# 📂 File paths
equity_file = os.path.join(save_path, "tcs_equity_data.csv")
future_file = os.path.join(save_path, "tcs_futures_data.csv")
option_file = os.path.join(save_path, "tcs_options_data.csv")


In [ ]:
from scipy import stats

def add_relationship_metadata(combined):

    df = combined.copy()

    # =================================================
    # A. Equity-Futures Relationships
    # =================================================
    if all(col in df.columns for col in ['equity_close', 'futures_close']):
        # Basis calculations
        for pt in ['open', 'high', 'low', 'close']:
            if f'equity_{pt}' in df.columns and f'futures_{pt}' in df.columns:
                df[f'basis_{pt}_absolute'] = df[f'futures_{pt}'] - df[f'equity_{pt}']
                df[f'basis_{pt}_relative'] = df[f'basis_{pt}_absolute'] / df[f'equity_{pt}']

        # Convergence metrics
        if 'basis_close_absolute' in df.columns:
            df['basis_close_convergence'] = df['basis_close_absolute'].diff()
            df['basis_close_velocity'] = df['basis_close_convergence'].diff()

        df['futures_equity_ratio'] = df['futures_close'] / df['equity_close']

    # =================================================
    # B. Options-Spot Relationships
    # =================================================
    required_option_cols = ['options_close', 'options_strike_price', 'options_right']
    if all(col in df.columns for col in required_option_cols + ['equity_close']):
        call_mask = df['options_right'].str.lower() == 'call'

        # Moneyness
        df['options_moneyness'] = df['equity_close'] / df['options_strike_price']
        df['log_moneyness'] = np.log(df['options_strike_price'] / df['equity_close'])

        # Intrinsic/extrinsic
        df['options_intrinsic_value'] = np.where(
            call_mask,
            np.maximum(0, df['equity_close'] - df['options_strike_price']),
            np.maximum(0, df['options_strike_price'] - df['equity_close'])
        )
        df['options_extrinsic_value'] = df['options_close'] - df['options_intrinsic_value']
        df['options_extrinsic_pct'] = df['options_extrinsic_value'] / df['options_close']

    # =================================================
    # C. Options-Futures Relationships (NEWLY ADDED BACK)
    # =================================================
    if all(col in df.columns for col in ['futures_close'] + required_option_cols):
        df['options_futures_intrinsic'] = np.where(
            call_mask,
            np.maximum(0, df['futures_close'] - df['options_strike_price']),
            np.maximum(0, df['options_strike_price'] - df['futures_close'])
        )
        df['options_futures_extrinsic'] = df['options_close'] - df['options_futures_intrinsic']
        df['implied_carry_cost'] = (
            (df['futures_close'] - df['options_strike_price']) -
            (df['options_close'] * np.where(call_mask, 1, -1))
        )

    # =================================================
    # D. Volatility Relationships (Fixed Version)
    # =================================================
    if all(col in df.columns for col in ['options_close', 'equity_close', 'options_strike_price', 'options_expiry_date']):
        try:
            # Calculate time to expiry in years
            time_to_expiry = ((pd.to_datetime(df['options_expiry_date']) - df['datetime']).dt.days / 365)
            time_to_expiry = time_to_expiry.replace(0, np.nan)  # Avoid division by zero

            # Calculate simple implied volatility proxy
            df['options_implied_volatility'] = (
                df['options_close'] /
                df['equity_close'] /
                np.sqrt(time_to_expiry)
            )
        except Exception as e:
            print(f"Couldn't calculate implied vol: {str(e)}")

    # Keep the realized volatility calculation
    if 'equity_close' in df.columns:
        log_returns = np.log(df['equity_close'] / df['equity_close'].shift(1))
        df['realized_vol_5min'] = log_returns.rolling(12).std() * np.sqrt(252*78)

    # =================================================
    # E. Technical Indicator Relationships (NEWLY ADDED BACK)
    # =================================================
    for indicator in ['RSI_14', 'MACD', 'ADX_14']:
        if f'equity_{indicator}' in df.columns and f'futures_{indicator}' in df.columns:
            df[f'equity_futures_{indicator}_divergence'] = (
                df[f'equity_{indicator}'] - df[f'futures_{indicator}'])

    # Composite trend strength
    if all(col in df.columns for col in ['equity_ADX_14', 'futures_ADX_14']):
        df['composite_trend_strength'] = (
            0.6*df['equity_ADX_14'] +
            0.4*df['futures_ADX_14'])

    # =================================================
    # F. Time-Based Metrics (KEPT AS REQUESTED)
    # =================================================
    if 'options_expiry_date' in df.columns:
        df['days_to_expiry'] = (pd.to_datetime(df['options_expiry_date']) - df['datetime']).dt.days
        if 'options_extrinsic_value' in df.columns:
            df['daily_theta'] = df['options_extrinsic_value'] / df['days_to_expiry'].replace(0, np.nan)
        df['weekly_expiry'] = df['datetime'].dt.dayofweek == 3  # Thursday expiries

    # =================================================
    # G. Volume and Liquidity Metrics
    # =================================================
    # 1. Volume ratios
    if all(col in df.columns for col in ['equity_volume', 'futures_volume']):
        df['futures_equity_volume_ratio'] = df['futures_volume'] / df['equity_volume']

    if 'options_volume' in df.columns and 'equity_volume' in df.columns:
        df['options_equity_volume_ratio'] = df['options_volume'] / df['equity_volume']

    # 2. Open interest metrics
    if 'options_open_interest' in df.columns:
        df['options_oi_volume_ratio'] = df['options_open_interest'] / df['options_volume'].replace(0, np.nan)

    return df

def enhance_metrics(combined_df):
    """
    Add layers of sophisticated metrics to the combined dataframe
    """
    df = combined_df.copy()

    # ==============================================
    # A. Price-Based Metrics
    # ==============================================

    # 1. Extended Basis Calculations
    for price_type in ['open', 'high', 'low', 'close']:
        if f'equity_{price_type}' in df.columns and f'futures_{price_type}' in df.columns:
            df[f'basis_{price_type}_raw'] = df[f'futures_{price_type}'] - df[f'equity_{price_type}']
            df[f'basis_{price_type}_pct'] = df[f'basis_{price_type}_raw'] / df[f'equity_{price_type}']

    # 2. Price Ratios and Spreads
    df['futures_premium'] = df['futures_close'] / df['equity_close'] - 1
    df['overnight_gap'] = df['equity_open'] / df['equity_close'].shift(1) - 1

    # ==============================================
    # B. Options-Specific Metrics
    # ==============================================

    if 'options_strike_price' in df.columns:
        # 1. Advanced Moneyness Metrics
        df['log_moneyness'] = np.log(df['equity_close'] / df['options_strike_price'])
        df['forward_moneyness'] = df['futures_close'] / df['options_strike_price'] if 'futures_close' in df.columns else np.nan

        # 2. Synthetic Greeks (simplified)
        required_cols = ['log_moneyness', 'options_right', 'options_implied_volatility', 'equity_close', 'options_expiry_date', 'datetime']
        if all(col in df.columns for col in required_cols):

          # Time to expiry in years
          df['time_to_expiry'] = (pd.to_datetime(df['options_expiry_date']) - df['datetime']).dt.days / 365
          df['time_to_expiry'] = df['time_to_expiry'].clip(lower=1e-6)  # prevent division by zero

          # Convenience variables
          d1 = df['log_moneyness'] / (df['options_implied_volatility'] * np.sqrt(df['time_to_expiry']))
          d2 = d1 - df['options_implied_volatility'] * np.sqrt(df['time_to_expiry'])

          # Delta Proxy
          df['delta_proxy'] = np.where(
              df['options_right'] == 'call',
              stats.norm.cdf(d1),
              stats.norm.cdf(d1) - 1
              )

          # Gamma Proxy
          df['gamma_proxy'] = stats.norm.pdf(d1) / (df['equity_close'] * df['options_implied_volatility'] * np.sqrt(df['time_to_expiry']))

          # Vega Proxy (scaled to 1% vol change)
          df['vega_proxy'] = df['equity_close'] * stats.norm.pdf(d1) * np.sqrt(df['time_to_expiry']) / 100

        # 3. Probability Metrics
        df['itm_probability'] = np.where(df['options_right'] == 'call',
                                       stats.norm.cdf(df['log_moneyness']),
                                       stats.norm.cdf(-df['log_moneyness']))

        df['otm_probability'] = np.where(df['options_right'] == 'call',
                                        stats.norm.cdf(-df['log_moneyness']),
                                        stats.norm.cdf(df['log_moneyness']))

        df['atm_probability'] = 2 * np.abs(df['log_moneyness']) - df['itm_probability'] - df['otm_probability']

    # ==============================================
    # C. Volume and Liquidity Metrics
    # ==============================================

    # 1. Volume Ratios
    for inst in ['futures', 'options']:
        if f'{inst}_volume' in df.columns:
            df[f'{inst}_volume_ratio'] = df[f'{inst}_volume'] / df['equity_volume']
            df[f'{inst}_volatility_ratio'] = df[f'{inst}_volume'] / df[f'{inst}_close'].rolling(20).std()

    # 2. Liquidity Proxies
    if 'options_open_interest' in df.columns:
        df['options_liquidity_score'] = (df['options_volume'] * df['options_close']) / df['options_open_interest']

    # ==============================================
    # D. Technical Composite Metrics
    # ==============================================

    # 1. Divergence Scores
    for indicator in ['RSI_14', 'MACD', 'ATR_14']:
        for pair in [('equity', 'futures'), ('equity', 'options')]:
            if f'{pair[0]}_{indicator}' in df.columns and f'{pair[1]}_{indicator}' in df.columns:
                df[f'{pair[0]}_{pair[1]}_{indicator}_divergence'] = (
                    df[f'{pair[0]}_{indicator}'] - df[f'{pair[1]}_{indicator}'])

    # 2. Multi-Instrument Signals
    df['composite_trend_strength'] = (
        0.4*df['equity_ADX_14'] +
        0.3*df['futures_ADX_14'] +
        0.3*df['options_implied_volatility'])

    # 3. Market regime classification
    if all(col in df.columns for col in ['equity_RSI_14', 'futures_RSI_14']):
        conditions = [
            (df['equity_RSI_14'] > 70) & (df['futures_RSI_14'] > 70),
            (df['equity_RSI_14'] < 30) & (df['futures_RSI_14'] < 30)
        ]
        df['market_regime'] = np.select(conditions, ['overbought', 'oversold'], default='neutral')

    # ==============================================
    # E. Time-Based Metrics
    # ==============================================

    if 'options_expiry_date' in df.columns:
        # 1. Time Decay Metrics
        df['days_to_expiry'] = (pd.to_datetime(df['options_expiry_date']) - df['datetime']).dt.days
        df['theta_proxy'] = df['options_extrinsic_value'] / np.sqrt(df['days_to_expiry'])

        # 2. Expiry-Cluster Effects
        df['weekly_expiry'] = df['datetime'].dt.dayofweek == 3  # Thursday expiries

    # ==============================================
    # F. Risk Metrics
    # ==============================================

    # 1. Value-at-Risk Proxies
    for inst in ['equity', 'futures', 'options']:
        if f'{inst}_close' in df.columns:
            returns = df[f'{inst}_close'].pct_change(fill_method=None)
            df[f'{inst}_var_95'] = returns.rolling(20).apply(lambda x: np.percentile(x.dropna(), 5))

    # 2. Correlation Metrics
    if all(col in df.columns for col in ['equity_close', 'futures_close', 'options_close']):
        df['spot_futures_correlation'] = df['equity_close'].rolling(20).corr(df['futures_close'])

    # 3. Calculate cross-correlations (FIXED for lookahead bias)
    # FIXED: Use rolling correlation to avoid lookahead bias
    # Previous line was problematic: shift(1).corr() doesn't work as intended
    df['futures_equity_correlation_20'] = df['futures_close'].rolling(20).corr(df['equity_close'])
    
    # If you need lag correlation, use proper rolling lag correlation
    df['futures_equity_lag1_correlation_20'] = df['futures_close'].shift(1).rolling(20).corr(df['equity_close'])

    return df

def combine_all_data(equity_df, futures_df, options_df):

    from pandas.api.types import is_datetime64_any_dtype as is_datetime

    # Create copies to avoid modifying originals
    equity = equity_df.copy()
    futures = futures_df.copy()
    options = options_df.copy()

    # =====================================================================
    # 1. Standardize datetime across all datasets
    # =====================================================================
    for df in [equity, futures, options]:
        if 'datetime' in df.columns and not is_datetime(df['datetime']):
            df['datetime'] = pd.to_datetime(df['datetime'])

    # =====================================================================
    # 2. Process Equity Data (Base Timeline)
    # =====================================================================
    equity = equity.add_prefix('equity_')
    equity.rename(columns={'equity_datetime': 'datetime'}, inplace=True)

    # =====================================================================
    # 3. Process Futures Data (1:1 with Equity)
    # =====================================================================
    futures = futures.add_prefix('futures_')
    futures.rename(columns={'futures_datetime': 'datetime'}, inplace=True)

    # Keep only the nearest expiry contract (most liquid)
    if 'futures_expiry_date' in futures.columns:
        nearest_expiry = futures['futures_expiry_date'].min()
        futures = futures[futures['futures_expiry_date'] == nearest_expiry]

    # =====================================================================
    # 4. Process Options Data (Complex - Many-to-One with Equity)
    # =====================================================================
    options = options.add_prefix('options_')
    options.rename(columns={'options_datetime': 'datetime'}, inplace=True)

    # Extract option type (call/put) from symbol if right column doesn't exist
    if 'options_right' not in options.columns and 'options_symbol' in options.columns:
        options['options_right'] = options['options_symbol'].str[-1].str.lower().map({'c':'call', 'p':'put'})
    elif 'options_strike_price' in options.columns and 'options_symbol' in options.columns:
        # Alternative method to determine call/put if right isn't available
        options['options_right'] = options['options_symbol'].str.extract(r'([CP])$')[0].str.lower().map({'c':'call', 'p':'put'})
    else:
        raise ValueError("Cannot determine option type (call/put) - missing both 'right' and 'symbol' columns")

    # =====================================================================
    # 5. Multi-Level Combination Strategy
    # =====================================================================
    # First merge equity and futures (1:1 relationship)
    combined = pd.merge(
        equity,
        futures,
        on='datetime',
        how='outer',
        suffixes=('', '_futures')
    )

    # Then merge with options (many-to-one relationship)
    combined = pd.merge(
        combined,
        options,
        on='datetime',
        how='outer',
        suffixes=('', '_options')
    )

    # =====================================================================
    # 6. Column Organization and Cleanup
    # =====================================================================
    # Reorder columns logically
    base_cols = ['datetime']

    equity_cols = [c for c in combined.columns
                  if c.startswith('equity_') and c not in base_cols]

    futures_cols = [c for c in combined.columns
                  if c.startswith('futures_') and c not in base_cols]

    options_cols = [c for c in combined.columns
                  if c.startswith('options_') and c not in base_cols]

    # Final column order
    column_order = (
        base_cols +
        sorted(equity_cols) +
        sorted(futures_cols) +
        sorted(options_cols)
    )

    combined = combined[column_order]

    # =====================================================================
    # 7. Add Relationship Metadata
    # =====================================================================
    combined = add_relationship_metadata(combined)

    # Add expiration countdown for derivatives
    for col in ['futures_expiry_date', 'options_expiry_date']:
        if col in combined.columns:
            combined[f'{col}_days_remaining'] = (
                pd.to_datetime(combined[col]) - combined['datetime']
            ).dt.days

    return combined

try:
    master_df = combine_all_data(equity_df, future_df, option_df)
    print("✅ Successfully combined data!")
except Exception as e:
    print(f"❌ Error combining data: {str(e)}")

# Then enhance with additional metrics
enhanced_df = enhance_metrics(master_df)
print("✅ Successfully enhanced data!.")

import pandas as pd
import numpy as np

# Ensure datetime is index and sorted
enhanced_df.index = pd.to_datetime(enhanced_df.index)
enhanced_df = enhanced_df.sort_index()

# Replace infinite values with NaNs, then fill
enhanced_df.replace([np.inf, -np.inf], np.nan, inplace=True)
enhanced_df.ffill(inplace=True)
enhanced_df.bfill(inplace=True)

# Identify numeric columns only (skip string/object types like stock_code)
numeric_cols = enhanced_df.select_dtypes(include=[np.number]).columns.tolist()

# Filter equity/futures/options to only numeric indicators
equity_cols = [col for col in numeric_cols if col.startswith("equity_")]
futures_cols = [col for col in numeric_cols if col.startswith("futures_")]
options_cols = [col for col in numeric_cols if col.startswith("options_")]

# Strip prefixes to find matching indicator names
equity_indicators = set(col.replace("equity_", "") for col in equity_cols)
futures_indicators = set(col.replace("futures_", "") for col in futures_cols)
options_indicators = set(col.replace("options_", "") for col in options_cols)

# Common indicators
shared_futures = equity_indicators & futures_indicators
shared_options = equity_indicators & options_indicators

# Rolling windows
rolling_windows = [5, 10, 20, 50]

# Function to calculate rolling correlation (FIXED for lookahead bias)
def compute_rolling_corr_fixed(df, col1, col2, window):
    """
    Calculate rolling correlation ensuring no lookahead bias.
    Uses only historical data for each timestamp.
    """
    # The default pandas rolling().corr() is correct - it uses only past data
    # But we add explicit min_periods to ensure robust calculation
    return df[col1].rolling(window=window, min_periods=window//2).corr(df[col2])

# Collect all new columns in a dictionary first
new_corr_cols = {}

print("🛡️ Computing rolling correlations with lookahead bias protection...")

for indicator in shared_futures:
    eq_col = f"equity_{indicator}"
    fut_col = f"futures_{indicator}"
    if eq_col in enhanced_df.columns and fut_col in enhanced_df.columns:
        for win in rolling_windows:
            new_col = f"corr_equity_futures_{indicator}_win{win}"
            new_corr_cols[new_col] = compute_rolling_corr_fixed(enhanced_df, eq_col, fut_col, win)

for indicator in shared_options:
    eq_col = f"equity_{indicator}"
    opt_col = f"options_{indicator}"
    if eq_col in enhanced_df.columns and opt_col in enhanced_df.columns:
        for win in rolling_windows:
            new_col = f"corr_equity_options_{indicator}_win{win}"
            new_corr_cols[new_col] = compute_rolling_corr_fixed(enhanced_df, eq_col, opt_col, win)

# Efficiently join all new columns at once
enhanced_df = pd.concat([enhanced_df, pd.DataFrame(new_corr_cols, index=enhanced_df.index)], axis=1)

print("✅ Cleaned non-numeric columns and added rolling correlation features efficiently.")

def add_all_returns(df, price_columns, rolling_windows=[3, 5, 10], risk_window=20):
    df = df.copy()  # Prevent modifying original

    for col in price_columns:
        # Skip if column doesn't exist
        if col not in df.columns:
            continue

        # Simple Return
        df[f'{col}_return'] = df[col].pct_change()

        # Log Return — add safe check for zero/negative
        with np.errstate(divide='ignore', invalid='ignore'):
            log_return = np.log(df[col] / df[col].shift(1))
            log_return.replace([np.inf, -np.inf], np.nan, inplace=True)
            df[f'{col}_log_return'] = log_return

        # Rolling Returns
        for w in rolling_windows:
            df[f'{col}_rolling_return_{w}'] = df[col].pct_change(periods=w)

        # Risk-adjusted return (Sharpe proxy)
        returns = df[col].pct_change()
        rolling_mean = returns.rolling(risk_window).mean()
        rolling_std = returns.rolling(risk_window).std()
        sharpe_like = rolling_mean / rolling_std
        sharpe_like.replace([np.inf, -np.inf], np.nan, inplace=True)
        df[f'{col}_risk_adjusted_return'] = sharpe_like

    # Optionally clean up all NaNs and Infs
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.bfill(inplace=True)
    df.ffill(inplace=True)  # Or use df.fillna(method='bfill'), etc.

    return df

price_cols = [
    'equity_close', 'equity_high', 'equity_low', 'equity_open', 'equity_volume',
    'futures_close', 'futures_high', 'futures_low', 'futures_open', 'futures_volume',
    'options_close', 'options_high', 'options_low', 'options_open', 'options_volume'
]
enhanced_df = add_all_returns(enhanced_df, price_cols)
print(f"✅ Successfully added returns!")

# --- SETTINGS ---
correlation_threshold = 0.95
target_time_windows = [1, 5, 10, 15, 30]
target_thresholds = [0.005, 0.0075, 0.01, 0.015, 0.02, 0.25, 0.03, 0.05, 0.075, 0.10, 0.15, 0.20]  # 0.5%, 1%, 2%

# --- 1. CORRELATION FILTERING ---
def correlation_filter(df, columns, threshold):
    corr = df[columns].corr().abs()
    upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))
    drop_cols = [col for col in upper.columns if any(upper[col] > threshold)]
    return [col for col in columns if col not in drop_cols], drop_cols

filtered_price_cols, dropped_price_cols = correlation_filter(enhanced_df, price_cols, correlation_threshold)
print("Dropped highly correlated price columns:", dropped_price_cols)

# --- 2. TARGET GENERATION (FIXED FOR LOOKAHEAD BIAS) ---
def generate_price_targets_fixed(df, columns, time_windows, thresholds):
    """
    Generate price targets WITHOUT lookahead bias.
    
    CRITICAL FIX: This version removes the problematic shift(-win) operation
    that was causing lookahead bias in the original implementation.
    """
    print("🛡️ Generating targets with lookahead bias protection...")
    
    all_new_cols = []

    for col in columns:
        for win in time_windows:
            # CORRECT: Calculate future returns for target generation
            # This is acceptable for targets, but these must not be used as features
            current_prices = df[col]
            future_prices = df[col].shift(-win)  # OK for targets only
            
            # Calculate returns properly
            future_return = (future_prices - current_prices) / current_prices
            
            # Create target column names with clear indication these are targets
            target_ret_col = f'{col}_TARGET_ret_{win}periods'
            target_logret_col = f'{col}_TARGET_logret_{win}periods'

            # Clip values to prevent extreme outliers
            future_return_clipped = future_return.replace([np.inf, -np.inf], np.nan).clip(lower=-0.999)
            log_return = np.log1p(future_return_clipped.fillna(0))

            new_cols = {
                target_ret_col: future_return,
                target_logret_col: log_return
            }

            # Classification labels (these are also targets)
            for thresh in thresholds:
                thresh_str = str(thresh).replace('.', '_')
                up = (future_return > thresh).astype(int)
                down = (future_return < -thresh).astype(int)
                neutral = ((future_return <= thresh) & (future_return >= -thresh)).astype(int)

                new_cols[f'{col}_TARGET_up_{win}p_{thresh_str}'] = up
                new_cols[f'{col}_TARGET_down_{win}p_{thresh_str}'] = down
                new_cols[f'{col}_TARGET_neutral_{win}p_{thresh_str}'] = neutral

            # Append to collection
            all_new_cols.append(pd.DataFrame(new_cols, index=df.index))

    # Concatenate all new columns
    if all_new_cols:
        new_features_df = pd.concat(all_new_cols, axis=1)
        df = pd.concat([df, new_features_df], axis=1)

    # CRITICAL: Remove the last max(time_windows) rows as they will have NaN targets
    max_window = max(time_windows)
    original_length = len(df)
    
    if len(df) > max_window:
        df = df[:-max_window].copy()
        print(f"🛡️ LOOKAHEAD BIAS PROTECTION: Removed last {max_window} rows")
        print(f"   Dataset length: {original_length} → {len(df)}")
        print(f"   This prevents using future data in training!")

    # Clean up infinite values and NaNs
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    
    # For target columns, keep NaN values (don't fill with 0)
    # For feature columns, can fill with appropriate methods
    target_cols = [col for col in df.columns if 'TARGET' in col]
    feature_cols = [col for col in df.columns if 'TARGET' not in col]
    
    # Fill feature columns only
    if feature_cols:
        df[feature_cols] = df[feature_cols].fillna(method='ffill').fillna(method='bfill')
    
    df = df.copy()  # defragment

    print(f"✅ Target generation completed with {len(target_cols)} target columns")
    print(f"⚠️  Remember: TARGET columns are for prediction only, not features!")

    return df

enhanced_df = generate_price_targets_fixed(enhanced_df, filtered_price_cols, target_time_windows, target_thresholds)

print(f"✅ Successfully Added Bias-Free Price Targets!")